## Gemma 2 FineTuning 

In [1]:
### 기본 설치 모듈 ###
! pip install -q -U transformers datasets bitsandbytes peft trl accelerate

## 데이터베이스 불러오기

In [1]:
# 데이터셋 로드 : KULLM이라는 LLM의 학습 데이터셋 불러오기
from datasets import load_dataset

dataset = load_dataset('nlpai-lab/kullm-v2', split="train")
print("데이터 셋 종류 :", dataset.features.keys())
print("id 종류 :", set(dataset['id']))
for key in list(set(dataset['id'])):
    print(f"{str(key)} 퍼센트 :", round(len([i for i in dataset if i['id'] ==str(key)])/len(dataset) * 100, 2), '%')

/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


데이터 셋 종류 : dict_keys(['id', 'instruction', 'input', 'output'])
id 종류 : {'vicuna_{idx}', 'alpaca_{idx}', 'dolly_{idx}'}
vicuna_{idx} 퍼센트 : 56.1 %
alpaca_{idx} 퍼센트 : 34.07 %
dolly_{idx} 퍼센트 : 9.83 %


In [2]:
dataset[-1]

{'id': 'vicuna_{idx}',
 'instruction': '항바이러스제는 생명을 위협하는 바이러스에만 사용되나요?',
 'input': '',
 'output': '항바이러스제는 생명을 위협하는 바이러스에만 사용되는 것이 아닙니다. 경증 및 중증 바이러스를 포함한 광범위한 바이러스 감염을 치료하는 데 사용할 수 있습니다. 항바이러스제 사용 여부는 일반적으로 특정 바이러스, 감염의 중증도, 환자의 전반적인 건강 상태에 따라 결정됩니다.예를 들어, 항바이러스제는 인플루엔자(독감) 치료에 자주 사용되는데, 인플루엔자는 노인, 어린이, 특정 기저질환이 있는 사람 등 특정 사람들에게는 심각하고 심지어 생명을 위협하는 질병이 될 수 있습니다. 항바이러스제는 독감의 중증도와 기간을 줄이는 데 도움이 될 수 있으며, 폐렴과 같은 심각한 합병증을 예방하는 데도 도움이 될 수 있습니다.다른 경우에는 항바이러스제를 사용하여 생명을 위협하지는 않지만 심각한 불편함을 유발하고 일상 활동에 지장을 줄 수 있는 바이러스 감염을 치료할 수 있습니다. 예를 들어, 항바이러스제는 피부나 점막에 통증성 궤양을 유발할 수 있는 단순포진 바이러스(HSV) 감염을 치료하는 데 사용될 수 있습니다.전반적으로 바이러스 감염을 치료하기 위해 항바이러스제를 사용할지 여부는 특정 바이러스, 감염의 심각성, 치료의 잠재적 위험과 이점을 포함한 다양한 요인에 따라 결정됩니다.'}

## Modul Loading

In [3]:
# 모듈 정리 
import torch
from transformers import AutoModelForCausalLM, GemmaTokenizerFast
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# gemma 2 2b-it 모델을 기반하여 학습 시킨다.
BASE_MODEL = "google/gemma-2-2b-it"

### 모델 실행 시 주의 한 번만 실행 요망 ###
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0})
tokenizer = GemmaTokenizerFast.from_pretrained(BASE_MODEL)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.65s/it]


In [4]:
tokenizer.encode("Hello this is a test")

[2, 4521, 736, 603, 476, 2121]

In [5]:
# chat 형식을 가져왔기 때문에 모델은 <end_of_turn> 구분자 뒤로 또다른 내용을 생성하려고 할 수 있습니다.
# 이런 경우 쓸데 없는 내용들이 계속 이어져 출력이 될 수 있습니다.
def generate_prompts(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        messages = [
            {"role": "user",
             "content": "사용자의 질문 입니다. 적절한 답변을 해주세요:\n\n {}".format(example['instruction'][i])}, 
            {"role": "assistant",
             "content": "{}".format(example['output'][i])}
        ]
        chat_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        print(chat_message)
        output_texts.append(chat_message)

    return output_texts

# 문장의 끝이라는 <eos> token을 명시적으로 붙여서 학습
# 코드 이쁘게 만든버전
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['instruction'])):
        prompt_list.append (
            f"<bos><start_of_turn>user\n"
            "사용자의 질문 입니다. 적절한 답변을 해주세요:\n\n"
            f"{example['instruction'][i]}<end_of_turn>\n"
            "<start_of_turn>model\n"
            f"{example['output'][i]}<end_of_turn><eos>"
        )
    
    return prompt_list


In [6]:
generate_prompts(dataset[:1]), generate_prompt(dataset[:1])

<bos><start_of_turn>user
사용자의 질문 입니다. 적절한 답변을 해주세요:

 3원색이란 무엇인가요?<end_of_turn>
<start_of_turn>model
세 가지 기본 색은 빨강, 파랑, 노랑입니다. 이 색은 다른 색을 혼합하여 만들 수 없고 다른 모든 색은 다양한 비율로 조합하여 만들 수 있기 때문에 원색이라고 부릅니다. 빛에 사용되는 첨가제 색상 시스템에서 원색은 빨강, 녹색, 파랑(RGB)입니다.<end_of_turn>



(['<bos><start_of_turn>user\n사용자의 질문 입니다. 적절한 답변을 해주세요:\n\n 3원색이란 무엇인가요?<end_of_turn>\n<start_of_turn>model\n세 가지 기본 색은 빨강, 파랑, 노랑입니다. 이 색은 다른 색을 혼합하여 만들 수 없고 다른 모든 색은 다양한 비율로 조합하여 만들 수 있기 때문에 원색이라고 부릅니다. 빛에 사용되는 첨가제 색상 시스템에서 원색은 빨강, 녹색, 파랑(RGB)입니다.<end_of_turn>\n'],
 ['<bos><start_of_turn>user\n사용자의 질문 입니다. 적절한 답변을 해주세요:\n\n3원색이란 무엇인가요?<end_of_turn>\n<start_of_turn>model\n세 가지 기본 색은 빨강, 파랑, 노랑입니다. 이 색은 다른 색을 혼합하여 만들 수 없고 다른 모든 색은 다양한 비율로 조합하여 만들 수 있기 때문에 원색이라고 부릅니다. 빛에 사용되는 첨가제 색상 시스템에서 원색은 빨강, 녹색, 파랑(RGB)입니다.<end_of_turn><eos>'])

## Finetuning 

In [7]:
from transformers import BitsAndBytesConfig
from peft import LoraConfig
import torch

# LoRA 설정: 대규모 언어 모델의 특정 레이어에서만 파라미터를 미세 조정하여
# 메모리 사용량을 줄이고 학습 효율성을 높임
lora_config = LoraConfig(
    r=6,  
    lora_alpha=8, 
    lora_dropout=0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],  
    task_type="CAUSAL_LM",
)

# BitsAndBytes 설정: 모델을 4비트로 양자화하여 메모리 사용량을 줄이고 성능 최적화
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.float16
)

In [8]:
from transformers import AutoModelForCausalLM, GemmaTokenizerFast

# gemma 2 2b-it 모델을 기반하여 학습 시킨다.
BASE_MODEL = "google/gemma-2-2b-it"

tokenizer = GemmaTokenizerFast.from_pretrained(BASE_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto"  # 자동으로 GPU에 할당
)
tokenizer.padding_side = 'right'

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.70s/it]


In [9]:
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
#        num_train_epochs = 1,
        max_steps=3000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps = 1000,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Map: 100%|██████████| 152630/152630 [00:35<00:00, 4294.81 examples/s]
/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precisi

In [10]:
### 학습 버튼 (딸깍) ### 
trainer.train()

It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Step,Training Loss
100,2.228500
200,1.657000
300,1.512600
400,1.468700
500,1.446300
600,1.442700
700,1.406300
800,1.428100
900,1.438100
1000,1.432300


TrainOutput(global_step=3000, training_loss=1.4302204437255859, metrics={'train_runtime': 7910.5828, 'train_samples_per_second': 1.517, 'train_steps_per_second': 0.379, 'total_flos': 5.066762968044288e+16, 'train_loss': 1.4302204437255859, 'epoch': 0.07862150298106532})

## FT 저장하기

In [11]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL) # LoRA 저장

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

FT_MODEL_NAME = "gemma-2-2b-ko"
model = model.merge_and_unload()
model.save_pretrained(FT_MODEL_NAME)

trainer.tokenizer.save_pretrained(FT_MODEL_NAME) # Tokenizer 저장


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.11it/s]


In [2]:
! ls -alh lora_adapter
! ls -alh ./gemma-2-2b-ko

total 51M
drwxrwxr-x 2 ubuntu ubuntu 4.0K Sep 25 10:11 .
drwxrwxr-x 8 ubuntu ubuntu 4.0K Sep 25 10:12 ..
-rw-rw-r-- 1 ubuntu ubuntu 5.0K Sep 25 10:11 README.md
-rw-rw-r-- 1 ubuntu ubuntu  721 Sep 25 10:11 adapter_config.json
-rw-rw-r-- 1 ubuntu ubuntu  30M Sep 25 10:11 adapter_model.safetensors
-rw-rw-r-- 1 ubuntu ubuntu  636 Sep 25 10:11 special_tokens_map.json
-rw-rw-r-- 1 ubuntu ubuntu  17M Sep 25 10:11 tokenizer.json
-rw-rw-r-- 1 ubuntu ubuntu 4.1M Sep 25 10:11 tokenizer.model
-rw-rw-r-- 1 ubuntu ubuntu  46K Sep 25 10:11 tokenizer_config.json
total 4.9G
drwxrwxr-x 2 ubuntu ubuntu 4.0K Sep 25 10:12 .
drwxrwxr-x 8 ubuntu ubuntu 4.0K Sep 25 10:12 ..
-rw-rw-r-- 1 ubuntu ubuntu  880 Sep 25 10:12 config.json
-rw-rw-r-- 1 ubuntu ubuntu  187 Sep 25 10:12 generation_config.json
-rw-rw-r-- 1 ubuntu ubuntu 4.7G Sep 25 10:12 model-00001-of-00002.safetensors
-rw-rw-r-- 1 ubuntu ubuntu 230M Sep 25 10:12 model-00002-of-00002.safetensors
-rw-rw-r-- 1 ubuntu ubuntu  24K Sep 25 10:12 model.safetenso

## 모델 불러오기 및 추론

In [1]:
from transformers import GemmaTokenizerFast, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import pipeline
import torch

FINETUNE_MODEL = "gemma-2-2b-ko"
BASE_MODEL = "google/gemma-2-2b-it"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = GemmaTokenizerFast.from_pretrained(BASE_MODEL)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 모델을 4비트로 로드하여 메모리 효율성을 극대화함
    bnb_4bit_quant_type="nf4",  # NF4(Normalized Float 4) 방식의 4비트 양자화 사용
    bnb_4bit_compute_dtype=torch.float16  # 계산에 사용할 데이터 타입을 float16으로 설정 (16비트 부동소수점)
)

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)

/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.69s/it]


In [4]:
text = '오늘의 날씨는 맑았어 좋은 점이 뭘까?'
messages = [
    {
        "role": "user",
        "content": "사용자의 질문 입니다. 적절한 답변을 해주세요:\n\n{}".format(text)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

오늘의 날씨가 맑으면 햇빛이 쏟아지는 것으로 인해 기분이 좋고 활력을 얻을 수 있습니다. 또한 맑은 날씨는 산책이나 야외 활동을 즐기기 좋은 날이 될 수 있습니다.


## GGUF 파일로 변환 하기

In [2]:
# gguf 저장 깃허브 
! git clone https://github.com/ggerganov/llama.cpp.git

Cloning into 'llama.cpp'...
remote: Enumerating objects: 34659, done.
remote: Counting objects: 100% (6648/6648), done.
remote: Compressing objects: 100% (265/265), done.
remote: Total 34659 (delta 6513), reused 6406 (delta 6383), pack-reused 28011 (from 1)
Receiving objects: 100% (34659/34659), 57.99 MiB | 18.87 MiB/s, done.
Resolving deltas: 100% (25153/25153), done.


In [3]:
# gguf 변환
! python llama.cpp/convert_hf_to_gguf.py gemma-2-2b-ko

INFO:hf-to-gguf:Loading model: gemma-2-2b-ko
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
INFO:hf-to-gguf:token_embd.weight,                 torch.float16 --> F16, shape = {2304, 256000}
INFO:hf-to-gguf:blk.0.attn_norm.weight,            torch.float16 --> F32, shape = {2304}
INFO:hf-to-gguf:blk.0.ffn_down.weight,             torch.float16 --> F16, shape = {9216, 2304}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,             torch.float16 --> F16, shape = {2304, 9216}
INFO:hf-to-gguf:blk.0.ffn_up.weight,               torch.float16 --> F16, shape = {2304, 9216}
INFO:hf-to-gguf:blk.0.post_attention_norm.weight,  torch.float16 --> F32, shape = {2304}
INFO:hf-to-gguf:blk.0.post_ffw_norm.weight,        torch.float16 --> F32, shape = {2304}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,     

## 허깅페이스 업로드

In [1]:
! pip install ipywidgets huggingface_hub

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="gemma-2-2b-ko", push_to_hub=True)